In [88]:
import numpy as np
import math
import pickle
import pandas as pd 
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

read file

In [3]:
file = open('covid_dataset.pkl', 'rb')
checkpoint = pickle.load(file)
file.close()

make data sets

In [110]:
def norm(X):
    scaler = MinMaxScaler()
    scaler.fit(X)
    return scaler.transform(X)

def apply_norm_fix(X, should_norm):
    X = np.array(X, dtype=np.float64)
    imp = IterativeImputer(max_iter=1000, random_state=0)
    imp.fit(X)
    X = imp.transform(X)
    if should_norm:
        X = norm(X)
    return X

X_train  = apply_norm_fix(checkpoint["X_train"], True)
y_train = np.array(checkpoint["y_train_log_pos_cases"], dtype=np.float64)

#keep = np.isfinite(X_train).all(axis=1)

#X_train = X_train[keep]
#y_train = y_train[keep]

X_val = apply_norm_fix(checkpoint["X_val"], True)
y_val = np.array(checkpoint["y_val_log_pos_cases"], dtype=np.float64)

#keep = np.isfinite(X_val).all(axis=1)

#X_val = X_val[keep]
#y_val = y_val[keep]

big_X = apply_norm_fix(np.array(list(X_train) + list(X_val)), False)
big_y = np.array(list(y_train) + list(y_val), dtype=np.float64)

#keep = np.isfinite(big_X).all(axis=1)

#big_X = big_X[keep]
#big_y = big_y[keep]

X_test = apply_norm_fix(checkpoint["X_test"], True)

Use algorithm

In [223]:
params = {'learning_rate': np.linspace(0.001, 1, 100000, dtype=np.float64), 'n_estimators': np.linspace(1, 1000, 1000, dtype=np.int32)}

algorithm = RandomizedSearchCV(GBR(), params, n_jobs=-1)
x = X_train
y = y_train
model = algorithm.fit(x, y)
preds = model.predict(X_val)
mse = mean_squared_error(y_val, preds)
print(mse)

test_pred = model.predict(X_test)
pd.DataFrame(test_pred).to_csv("predictions.csv", header=["cases"], index_label="id")

2.181926899543532


In [224]:
print(model.best_params_)

{'n_estimators': 662, 'learning_rate': 0.16349896498964991}
